In [ ]:
import requests
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI
from langchain.tools import Tool


# 定義查詢 Wikidata 的函式
def query_wikidata(label: str) -> str:
    query = f"""
    SELECT ?qid WHERE {{
      ?qid rdfs:label "{label}"@zh .
    }}
    LIMIT 1
    """
    url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    response = requests.get(url, params={"query": query}, headers=headers)
    data = response.json()
    try:
        return data["results"]["bindings"][0]["qid"]["value"]
    except IndexError:
        return "找不到結果"

# 定義 SPARQL 查詢工具並加上 name
wiki_sparql_tool = Tool(
    name="Wikidata QID 查詢",  # 設定工具名稱
    func=lambda q: query_wikidata(q),  # 查詢 Wikidata 的函式
    description="輸入一個中文名稱，回傳對應的 Wikidata QID"
)


def local_query_sparql(query: str) -> str:
    headers = {
        'Accept': 'application/json',  # 接收 JSON 格式的回應
    }
    
    #     # 定義本地命名空間
    # local_namespace = "PREFIX ex: <http://example.org/schema#>\n"
    
    # # 將命名空間加入查詢語句
    # full_query = local_namespace + query
    
    
    # 發送 SPARQL 查詢到端點
    response = requests.get("http://192.168.133.39:8890/sparql", headers=headers, params={'query': query})
    
    if response.status_code == 200:
        # 如果查詢成功，解析回應並提取所需的結果
        results = response.json()
        if 'results' in results:
            bindings = results['results']['bindings']
            if bindings:
                # 假設您只需要第一條記錄的結果
                return bindings[0]['label']['value']
            else:
                return "未找到匹配的結果"
        else:
            return "查詢結果格式錯誤"
    else:
        return f"查詢失敗，錯誤代碼：{response.status_code}"

# 定義 SPARQL 查詢工具
local_sparql_tool = Tool(
    name="本地 SPARQL 查詢工具",  # 設定工具名稱
    func=local_query_sparql,  # 使用上面定義的查詢函數
    description="使用 SPARQL 查詢本地知識圖譜"
)



In [ ]:
from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
#os.environ["GROQ_API_KEY"] = ''
#API#2
#s.environ["GROQ_API_KEY"] = ''
#API#3
#os.environ["GROQ_API_KEY"] = ''
#API#4
os.environ["GROQ_API_KEY"] = ''
#API#5
#os.environ["GROQ_API_KEY"] = ''

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],

    model_name="llama-3.3-70b-versatile",       

    temperature=0.0,
  #  max_tokens=1000,
)

In [34]:
import langchain
# 設定 debug 模式為 False
#langchain.debug = True
langchain.debug = False

print(langchain.__version__)

0.3.14


In [29]:
# 初始化 Agent，這裡設置用於查詢的 SPARQL 工具
from langchain.agents import initialize_agent, AgentType
#from langchain.llms import OpenAI

#llm = OpenAI(temperature=0)

# 初始化 agent
agent = initialize_agent(
    tools=[local_sparql_tool, wiki_sparql_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 使用 agent 查詢 Wikidata QID
response = agent.run("請查詢 '蔣中正' 的 Wikidata QID")
print(response)



> Entering new AgentExecutor chain...
Thought: 我需要使用 Wikidata QID 查詢工具來查詢 '蔣中正' 的 Wikidata QID。
Action: Wikidata QID 查詢
Action Input: 蔣中正
Observation: http://www.wikidata.org/entity/Q16574
Thought:Thought: 我已經得到了 '蔣中正' 的 Wikidata QID，現在我可以直接回傳這個結果。
Final Answer: Q16574

> Finished chain.
Q16574


In [39]:

# 初始化 Agent，將 SPARQL 查詢工具傳遞給 Agent
agent = initialize_agent(
    tools=[sparql_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 使用 agent 查詢本地知識圖譜

    #     # 定義本地命名空間
local_namespace = "請定義本地命名空間PREFIX ex: <http://example.org/schema#>"
# # 將命名空間加入查詢語句
query = "請查詢'李登輝'的相關資訊"

  
response = agent.run(local_namespace + query)
print(response)





> Entering new AgentExecutor chain...
Thought: 首先，我需要定義本地命名空間PREFIX ex: <http://example.org/schema#>，然後我可以使用SPARQL查詢工具查詢'李登輝'的相關資訊。

Action: SPARQL 查詢工具
Action Input: PREFIX ex: <http://example.org/schema#> SELECT ?s ?p ?o WHERE { ?s ex:name "李登輝" . ?s ?p ?o . }
Observation: 未找到匹配的結果
Thought:Thought: 由於初始查詢未找到匹配的結果，我需要調整查詢條件。這次，我將嘗試查詢所有包含"李登輝"的實體。

Action: SPARQL 查詢工具
Action Input: PREFIX ex: <http://example.org/schema#> SELECT ?s ?p ?o WHERE { ?s ?p ?o . FILTER(contains(str(?o), "李登輝")) . }

KeyError: 'label'